In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Normalize the images to range [-1, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Define the classes
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# Define the data augmentation
datagen = ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    zoom_range=0.2
)

# Fit the data generator to the training data
datagen.fit(train_images)


In [ ]:
# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(6, (5, 5), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(120, activation='relu'))
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer=optimizers.SGD(learning_rate=0.001, momentum=0.9),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Define TensorBoard callback
!rm -rf logs/cifar10_experiment
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/cifar10_experiment")

# Train the model
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    epochs=2,
                    validation_data=(test_images, test_labels),
                    callbacks=[tensorboard_callback])

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Accuracy of the network on the 10000 test images: {test_acc:.2f}%')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/cifar10_experiment

In [ ]:
# Function to show an image using Matplotlib
def imshow(img, title=None):
    img = (img / 2) + 0.5  # unnormalize
    plt.imshow(img)
    if title is not None:
        plt.title(title)
    plt.show()

# Get some random training images and display them
for _ in range(4):
    img = train_images[np.random.randint(len(train_images))]
    imshow(img, title="Augmented Image")

In [ ]:
# Get some random test images
indices = np.random.choice(len(test_images), 4)
images, labels = test_images[indices], test_labels[indices]

# Make predictions
predictions = model.predict(images)

# Plot images with predictions
for i in range(4):
    plt.figure()
    plt.imshow(images[i])
    plt.title(f"GroundTruth: {class_names[labels[i][0]]} \nPredicted: {class_names[np.argmax(predictions[i])]}")
    plt.show()